In [37]:
# Importar Librerias
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from time import time

In [38]:
# Cargar todas los Data Sets
t = time()
train = pd.read_csv('datasets/train.csv',index_col=False) #OK
test = pd.read_csv('datasets/test.csv',index_col=False) #OK
event_type = pd.read_csv('datasets/event_type.csv',index_col=False) #OK
log_feature = pd.read_csv('datasets/log_feature.csv',index_col=False) #OK
resource_type = pd.read_csv('datasets/resource_type.csv',index_col=False) #OK
severity_type = pd.read_csv('datasets/severity_type.csv',index_col=False) 
print(str((time()-t)/60))

0.0016328096389770508


In [39]:
# Pivotear los Logs
## Output: x_log, x_max_log, x_log_count, x_log_order
t = time()
x_log = log_feature
x_log.log_feature.replace(' ','_',regex=True,inplace=True)
x_log = x_log.pivot_table(values='volume', index='id', 
                          columns='log_feature', aggfunc=[np.mean]).fillna(0).reset_index()
x_log['log_count'] = np.sum(x_log['mean'], axis=1)

# Encontar el mayor Volume por Id
x_max_log = log_feature.groupby(['id']).agg({'volume':'max'}).reset_index().rename(columns={'volume':'max_log'})

#Contar los Logs por Id
x_log_count = log_feature.groupby(['id']).agg({'log_feature':'count'}).reset_index().rename(columns={'log_feature':
                                                                                                     'log_count'})

# Mantener el Orden de los Id's
x_log_order = pd.DataFrame(log_feature['id'].unique()).rename(columns={0:'id'})
x_log_order['log_order'] = np.arange(1,len(x_log_order)+1)
print(str((time()-t)/60))
x_max_log.head(5)

0.00848313570022583


,id,max_log
0,1,2
1,2,1
2,3,2
3,4,3
4,5,11


In [40]:
# Ordenar las localizaciones y contarlas
## Output: x_panel, x_location

# Concatenar Train y Test
t=time()
x_panel = pd.concat([train, test])

# Contar localizaciones
x_location = x_panel.groupby(['location']).agg({'id':'count'}).reset_index().rename(columns={'id':'location_count'})

# Ordenar todo el Dataset (Train+Test) por el Nº de Log
x_panel = pd.merge(x_panel, x_log_order, on='id', how='left', sort=False)
x_panel.sort_values(by=['log_order'], inplace=True)
x_panel.reset_index(drop=True, inplace=True)
x_panel['location_order'] = 1

# Ordenar cada grupo de Localidades por Id
for i in list(x_panel['location'].unique()):
    x = np.arange(1,len(x_panel['location_order'].loc[x_panel['location']==i])+1)
    x_panel['location_order'].loc[x_panel['location']==i] = x
del x
print(str((time()-t)/60))

0.9331037243207295


In [41]:
# Pivotear los Eventos
## Output: x_event
t = time()
x_event = event_type
x_event.event_type.replace(' ','_',regex=True,inplace=True)
x_event = x_event.pivot_table(index='id', columns='event_type', aggfunc=len).fillna(0).reset_index()
x_event['event_count'] = np.sum(x_event[x_event.columns[1:len(x_event.columns)]], axis=1)
print(str((time()-t)/60))
x_event.head(5)

0.006348303953806559


event_type,id,event_type_1,event_type_10,event_type_11,event_type_12,event_type_13,event_type_14,event_type_15,event_type_17,event_type_18,...,event_type_50,event_type_51,event_type_52,event_type_53,event_type_54,event_type_6,event_type_7,event_type_8,event_type_9,event_count
0,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [42]:
# Pivotear Recursos
## Output: x_resource
t = time()
x_resource = resource_type
x_resource.resource_type.replace(' ','_',regex=True,inplace=True)
x_resource = x_resource.pivot_table(index='id', columns='resource_type', aggfunc=len).fillna(0).reset_index()
x_resource['resource_count'] = np.sum(x_resource[x_resource.columns[1:len(x_resource.columns)]], axis=1)
print(str((time()-t)/60))
x_resource.head(5)


0.004249846935272217


resource_type,id,resource_type_1,resource_type_10,resource_type_2,resource_type_3,resource_type_4,resource_type_5,resource_type_6,resource_type_7,resource_type_8,resource_type_9,resource_count
0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [63]:
# Pivotear Severidad
## Output: x_severity
t = time()
x_severity = severity_type
x_severity_count = x_severity.groupby(['severity_type']).agg({'id':'count'}).reset_index().rename(columns={'id':'severity_count'})
x_severity.severity_type.replace(' ','_',regex=True,inplace=True)
x_severity = x_severity.pivot_table(index='id', columns='severity_type', aggfunc=len).fillna(0).reset_index()
#x_severity = pd.merge(x_severity, x_severity_count, on='severity_type', how='left', sort=False)

print(str((time()-t)/60))
x_severity.head(5)


0.0032057722409566244


severity_type,id,severity_type_1,severity_type_2,severity_type_3,severity_type_4,severity_type_5
0,1,1.0,0.0,0.0,0.0,0.0
1,2,0.0,1.0,0.0,0.0,0.0
2,3,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,1.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0


In [44]:
# Realizar todos los Joins con el DataSet completo

# x_log
x_panel = pd.merge(x_panel, x_log, on='id', how='left', sort=False)

# x_max_log
x_panel = pd.merge(x_panel, x_max_log, on='id', how='left', sort=False)

# x_log_count
x_panel = pd.merge(x_panel, x_log_count, on='id', how='left', sort=False)

# x_location
x_panel = pd.merge(x_panel, x_location, on='location', how='left', sort=False)

# x_event
x_panel = pd.merge(x_panel, x_event, on='id', how='left', sort=False)

# x_resource
x_panel = pd.merge(x_panel, x_resource, on='id', how='left', sort=False)

# x_severity
x_panel = pd.merge(x_panel, x_severity, on='id', how='left', sort=False)


In [64]:
# Realizar Calculos adicionales sobre el DataSet
x_panel.location.replace('location ','',regex=True,inplace=True)
x_panel['location_ratio'] = x_panel['location_order'] / x_panel['location_count']
x_panel['max_proportion'] = x_panel['max_log'] / x_panel['log_count']
x_panel['f203_d_f82'] = x_panel[('mean','feature_203')] / x_panel[('mean','feature_82')]


# Separar el DataSet nuevamente en Train y Test
x_train = x_panel.loc[~x_panel['fault_severity'].isnull()].reset_index()
x_test = x_panel.loc[x_panel['fault_severity'].isnull()].reset_index()



(18552, 469)
Index([       'fault_severity',                    'id',
                    'location',             'log_order',
              'location_order',   ('mean', 'feature_1'),
        ('mean', 'feature_10'), ('mean', 'feature_100'),
       ('mean', 'feature_101'), ('mean', 'feature_102'),
       ...
              'resource_count',       'severity_type_1',
             'severity_type_2',       'severity_type_3',
             'severity_type_4',       'severity_type_5',
              'severity_count',        'location_ratio',
              'max_proportion',            'f203_d_f82'],
      dtype='object', length=469)


In [65]:
x_train.to_csv('py_train.csv', encoding='utf-8', index=False)
x_test.to_csv('py_test.csv', encoding='utf-8', index=False)